In [3]:
import pickle

single_run = None
with open('data/single_run.pkl', 'rb') as rf:
    single_run = pickle.load(rf)

with open('data/n_run.pkl', 'rb') as rf:
    n_runs = pickle.load(rf)

with open('data/n_run1.pkl', 'rb') as rf:
    n_runs1 = pickle.load(rf)

with open('single_run_pt.pkl', 'rb') as rf:
    single_run_pt = pickle.load(rf)

with open('n_run_pt.pkl', 'rb') as rf:
    n_runs_pt = pickle.load(rf)

In [4]:
import numpy as np
import time
from tabulate import tabulate
from n_run_evolutions import logmodel, X_train, Y_train, X_test, Y_test, accuracy_score, all_models_score_table

In [8]:
chromo_set_1, score_set_1 = single_run_pt

In [7]:
n_results = np.array(n_runs_pt)

C:\Users\student\AppData\Local\Temp\ipykernel_11924\125543596.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  n_results = np.array(n_runs_pt)


In [9]:
def predicted_sentiment_ratio(y_test, predictions):
    positive_percent = np.count_nonzero(predictions==1)*100//len(predictions)
    negative_percent = np.count_nonzero(predictions==0)*100//len(predictions)
    # print(f"Y_test: pos/neg percentage ~ {np.count_nonzero(y_test==1)*100//len(y_test)}, {np.count_nonzero(y_test==0)*100//len(y_test)}")
    return positive_percent, negative_percent

In [10]:
def test_accuracy(selected_genes):
    st = time.time()
    logmodel.fit(X_train[:, selected_genes], Y_train)
    et = time.time()

    predictions = logmodel.predict(X_test[:, selected_genes])
    ratio = predicted_sentiment_ratio(Y_test, predictions)
    return accuracy_score(Y_test, predictions), et-st, ratio

In [11]:
def pick_top_n_genes_n_run(genes, chromo_set, n=1):
    common_index = dict()
    chromo_len_in_each_gen = []
    for chromo in chromo_set:
        if n==1:
            chromo_len_in_each_gen.append(np.count_nonzero(chromo == 1))
            for i in range(len(chromo)):
                if chromo[i] == 1:
                    common_index[i] = common_index.setdefault(i, 0) + 1
        else:
            best_gen = np.array(chromo[:, -1])
            chromo_len_in_each_gen.append(np.count_nonzero(best_gen[0] == 1))
            for i in range(len(best_gen[0])):
                if best_gen[0][i] == 1:
                    common_index[i] = common_index.setdefault(i, 0) + 1

    if genes == -1:
        genes = len(common_index)

    sorted_common_index = np.array(sorted(
        common_index.items(), key=lambda x: x[1], reverse=True)[:genes])
    return sorted_common_index[:, 0], len(sorted_common_index), np.array(chromo_len_in_each_gen)

# Collect Common indexes from res of single run evolution


In [12]:
all_models_score_table.iloc[0, 1:]

Accuracy               0.8
Exec_Time_secs    1.973708
Name: 0, dtype: object

In [13]:
X_train.shape

(750, 1553)

# Collect Common indexes from res of n run evolution


In [14]:
n_results1[:, :, -1]

array([[array([0, 1, 0, ..., 1, 0, 0], dtype=int64), 0.764],
       [array([1, 0, 0, ..., 0, 0, 0], dtype=int64), 0.804],
       [array([0, 0, 0, ..., 1, 0, 0], dtype=int64), 0.804],
       [array([0, 1, 0, ..., 1, 1, 0], dtype=int64), 0.812],
       [array([0, 1, 0, ..., 1, 0, 0], dtype=int64), 0.78],
       [array([1, 1, 1, ..., 1, 0, 1], dtype=int64), 0.776],
       [array([0, 0, 0, ..., 1, 1, 1], dtype=int64), 0.748],
       [array([1, 0, 0, ..., 1, 0, 0], dtype=int64), 0.772],
       [array([1, 1, 0, ..., 0, 1, 0], dtype=int64), 0.788],
       [array([0, 0, 0, ..., 1, 0, 1], dtype=int64), 0.816],
       [array([0, 1, 0, ..., 0, 0, 0], dtype=int64), 0.776],
       [array([0, 0, 0, ..., 0, 0, 0], dtype=int64), 0.784],
       [array([1, 0, 1, ..., 0, 1, 0], dtype=int64), 0.78],
       [array([0, 0, 1, ..., 0, 0, 1], dtype=int64), 0.78],
       [array([0, 0, 0, ..., 1, 1, 1], dtype=int64), 0.748],
       [array([0, 0, 1, ..., 0, 0, 0], dtype=int64), 0.804],
       [array([1, 1, 1, ...

# Final Results

In [15]:
def tabulate_results(chromo_set, n):
    genes = [100, 150, 200, 250, -1]
    accuracy = []
    time_to_train = []
    sentiment_ratio = []
    for i in genes:
        top_n_genes, common_count, chromo_len = pick_top_n_genes_n_run(i, chromo_set, n)
        acc, ttt, sr = test_accuracy(top_n_genes)
        accuracy.append(acc)
        time_to_train.append(ttt)
        sentiment_ratio.append(sr)

    
    table_data = {'Genes': genes,
                'Accuracy': accuracy,
                'Time Taken to Train': time_to_train,
                'Sentiment Ratio(p/n)': sentiment_ratio
                }

    print('Actual chromosome length in X_Test: ', X_test.shape[1])
    print('RandomForest with full length -', all_models_score_table.iloc[0, 1:])
    print('Total Common genes count ~(-1): ', common_count)
    print(chromo_len)
    print(tabulate(table_data, headers='keys', tablefmt="simple_grid"))


Comparitive results for Single run evolution

In [16]:
tabulate_results(chromo_set_1, 1)

Actual chromosome length in X_Test:  1553
RandomForest with full length - Accuracy               0.8
Exec_Time_secs    1.973708
Name: 0, dtype: object
Total Common genes count ~(-1):  979
[100 100 100 100 100 100 208 208 224 224 224 181 181 228 277 255 242 242
 333 333 333 333 333 414 347 347 347 342 342 342]
  Genes    Accuracy    Time Taken to Train  Sentiment Ratio(p/n)
-------  ----------  ---------------------  ----------------------
    100       0.632               0.334103  (20, 79)
    150       0.728               0.329119  (33, 66)
    200       0.736               0.405903  (34, 66)
    250       0.764               0.411896  (41, 58)
     -1       0.784               0.88064   (48, 51)


Comparitive results for 30 run evolution

In [17]:
tabulate_results(n_results, 30)

Actual chromosome length in X_Test:  1553
RandomForest with full length - Accuracy               0.8
Exec_Time_secs    1.973708
Name: 0, dtype: object
Total Common genes count ~(-1):  1553
[395 376 530 366 506 376 394 423 359 409 580 341 475 350 473 382 549 469
 399 380 430 490 412 477 233 407 378 510 467 303]
  Genes    Accuracy    Time Taken to Train  Sentiment Ratio(p/n)
-------  ----------  ---------------------  ----------------------
    100       0.828               0.301193  (44, 55)
    150       0.828               0.316154  (44, 56)
    200       0.812               0.372003  (45, 54)
    250       0.824               0.424861  (46, 54)
     -1       0.792               1.10005   (46, 53)
